In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras import layers, models

def load_data(csv_path):
    df = pd.read_csv(csv_path, delimiter=',', header=None, skiprows=1, names=['filename', 'galley', 'island', 'l-shape', 'one-wall', 'u-shape'])
    filenames = df['filename'].values
    labels = df.iloc[:, 1:].values
    return df, filenames, labels

# Directory paths
train_dir = 'data/train'
test_dir = 'data/test'
validate_dir = 'data/valid'

# Load training data
train_csv_path = os.path.join(train_dir, '_classes.csv')
df_train, train_filenames, train_labels = load_data(train_csv_path)

# Load validation data
validate_csv_path = os.path.join(validate_dir, '_classes.csv')
df_valid, validate_filenames, validate_labels = load_data(validate_csv_path)

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_dataframe(dataframe=df_train,
                                                    directory=train_dir,
                                                    x_col='filename',
                                                    y_col=['galley', 'island', 'l-shape', 'one-wall', 'u-shape'],
                                                    target_size=(640, 640),
                                                    class_mode='other',
                                                    batch_size=32)

# Data augmentation for validation images
validate_datagen = ImageDataGenerator(rescale=1./255)

validate_generator = validate_datagen.flow_from_dataframe(dataframe=df_valid,
                                                          directory=validate_dir,
                                                          x_col='filename',
                                                          y_col=['galley', 'island', 'l-shape', 'one-wall', 'u-shape'],
                                                          target_size=(640, 640),
                                                          class_mode='other',
                                                          batch_size=32)

# Data augmentation for validation images
validate_datagen = ImageDataGenerator(rescale=1./255)

validate_generator = validate_datagen.flow_from_dataframe(dataframe=df_valid,
                                                          directory=validate_dir,
                                                          x_col='filename',
                                                          y_col=['galley', 'island', 'l-shape', 'one-wall', 'u-shape'],
                                                          target_size=(640, 640),
                                                          class_mode='other',
                                                          batch_size=32)

def create_baseline_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten and dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Input shape for color images (3 channels for RGB)
input_shape = (640, 640, 3)
num_classes = 5

# Create and compile the color model
baseline_model = create_baseline_model(input_shape, num_classes)
baseline_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
baseline_model.summary()

# Train the model with validation data
history = baseline_model.fit(train_generator, epochs=10, validation_data=validate_generator)

# Save the model
baseline_model.save('baseline_image_classification_model.h5')

## Improvements to Dataset

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="uvNrM5TfiqIdNezfmazG")
project = rf.workspace().project("shape-detection-using-image-classification")
model = project.version(2).model

# infer on a local image
print(model.predict("kitchen.jpg").json())

# save an image annotated with your predictions
model.predict("kitchen.jpg").save("prediction.jpg")